# Budget optimizer dashboard

In [278]:
# 
import pandas as pd
import sys
import json
import ipywidgets as widgets
encoding = sys.getdefaultencoding()
import os

import numpy as np

import math
import matplotlib.pyplot as plt

from scipy.optimize import curve_fit
import scipy
import scipy.stats as stat

import xlsxwriter
from pathlib import Path

import plotly.express as px
import plotly.graph_objects as go
pd.options.plotting.backend = "plotly"

from IPython.display import display

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

pd.set_option('display.max_columns', None)

## How to work with notebook

Basic settings:
* period: month, week, day
* Margin size: 1σ or 2σ
* Margin: upper, lower or zero boundary (optimistic, realistic  pessimistic)
* List of components we will consider for budget optimization (channel, platform and so on) 


Advanced settings:
* Filter: type, size, period (for outliers filtering)
* Error threshold in model? 


In [2]:
granularity = 'week'
margin_size = 1
margin_bound = 'upper' # 'mean' 'lower'
filter_type = 'avrg'   # 'median'
filter_length = 10     # number of points in sequence 
error_threshhold = 0   # I'm going to define it after optimisation task
zero_cross = True
independent_var = 'marketingInvestment'
dependent_var   = 'netRevenue'
precision = 3          # number of digits after point
dropped_ch = ['Not tracked', 'notset', 'unpaid'] # Channels to discard for platform level analysis 

In [3]:
#from BudgetOptimizer import EstimatorClass, settings_estimator
from Budget_optimiser_function import DateTimeID, usecols_, dtype_
#from Budget_optimiser_function import *

## Function's definition

In [266]:
# function for genetic algorithm to minimize (sum of squared error)
def sumOfSquaredError(parameterTuple):
    function = log_f # it's bad staff #TODO
    warnings.filterwarnings("ignore") # do not print warnings by genetic algorithm
    val = function(t_train, *parameterTuple)
    return np.sum((y_train - val) ** 2.0)

def select_agg_resample_df(df, index, granularity, use_nan):
    """Function for select and group  data in dataframe
    input: pandas dataframe, name of index, granularity['week','month'] default as in dataframe
    return: aggregated dataframe with extra column with indexes [ROI, CPI, CPC]"""
    group_df = df.groupby(['periodStartDate']).agg({
    'netRevenue': 'sum',
    'marketingInvestment': 'sum', 
    'visits': 'sum', 
    'conversions': 'sum',
    'deliveries': 'sum', 
    'impressions': 'sum', 
    'clicks': 'sum', 
    'grossProfit': 'sum' })   

    if granularity == 'week':
        group_agg_df = group_df.resample('W').agg('sum')
    elif granularity == 'month':
        group_agg_df = group_df.resample('M', convention='end').agg('sum')
    elif granularity == 'day':
        group_agg_df = group_df
    else:
        raise ValueError("Incorrect aggregation period, shuld be 'day', 'week' or 'month'")

    if use_nan == False:
        group_agg_df.fillna(0)
    else:
        pass
    return group_agg_df


from scipy.optimize import differential_evolution
import warnings
def generate_Initial_Parameters(t_train, y_train,function):
    # min and max used for bounds
    maxX = max(t_train)
    minX = min(t_train)
    maxY = max(y_train)
    minY = min(y_train)
    maxXY = max(maxX, maxY)

    parameterBounds = []
    parameterBounds.append([-maxXY, maxXY]) # seach bounds for a
    parameterBounds.append([-maxXY, maxXY]) # seach bounds for b
    #parameterBounds.append([-maxXY, maxXY]) # seach bounds for c

    # "seed" the numpy random number generator for repeatable results
    result = differential_evolution(sumOfSquaredError, parameterBounds, seed=3)
    return result.x


def regression_calc(df, function):
    """Calculate paramiters of regression function
    input: pandas dataframe [x,y]
    output: list[paramiters], list[std diviation err of paramiters]"""
    from scipy.optimize import curve_fit
    t_train = df[df.columns[0]]  #
    y_train = df[df.columns[1]]
    geneticParameters = generate_Initial_Parameters(t_train, y_train, log_f)
    popt, pcov = curve_fit(log_f, t_train, y_train, geneticParameters)
    #popt, pcov = curve_fit(function, t_train, y_train)
    error  = np.sqrt(np.diag(pcov))
    return popt, error

def filter_df(df, alpha):
    return df.ewm(alpha=alpha, adjust=False).mean()


# regression functions definition
def log_f(x, a, b):
    return a * (1-np.exp(-x/b)) #a * (1 - np.exp((x/b)))


def line_f(x, a, b):
    return a * x + b


def sine_f(x, a, b):
    return a * np.sin(b * x)


def show_chart(zzz_df):
    import plotly.graph_objects as go
    fig = go.Figure()
    # Add traces
    x=np.linspace(0, zzz_df.Max_week_invest.to_list()[0], num=100)
    fig.add_trace(
        go.Scatter(x=x ,
               y=log_f(x, zzz_df.a.to_list()[0], zzz_df.b.to_list()[0]),
               mode='lines',
               name='Regression line'),)

    fig.add_trace(
        go.Scatter(x=x ,
               y=log_f(x, zzz_df.a.to_list()[0], zzz_df.b.to_list()[0]) + 2*zzz_df.R_std.to_list()[0],
               mode='lines',
               name='Confidence interval 2sigma upper'))
    fig.add_trace(
        go.Scatter(x=x ,
               y=log_f(x, zzz_df.a.to_list()[0], zzz_df.b.to_list()[0])- 2*zzz_df.R_std.to_list()[0],
               mode='lines',
               name='Confidence interval 2sigma lower'))

    fig.add_trace(
        go.Scatter(x=zzz_df.Invest_prev_week,
               y=zzz_df.Prof_prev_week,
               mode='markers',
               name='Previous week'))
    fig.add_trace(
        go.Scatter(x=zzz_df.Invest_last_week,
               y=zzz_df.Prof_last_week,
               mode='markers',
               name='Current week (profit prediction)'))

    fig.update_layout(
        title_text=
        f"Profit to investment {zzz_df.Business_unit.to_list()[0]} on {zzz_df.Component.to_list()[0]} platform on {data_source.value} level with week granularity with {filter_type.value} filter",
    title_font=dict(size=16))
    fig.show()
    
def platform_calc(df):
    business_unit_code = df['businessUnit'].unique().tolist()
    business_unit_code.sort()
    channel_code_code = df['channelCode'].unique().tolist()
    channel_code_code.sort()
    currency_code_code = df['currencyCode'].unique().tolist()
    currency_code_code.sort()
    account_code = df['account'].unique().tolist()

    # selecl "time_window_regr" number of last weeks from dataset 

    df_time_selected = df[df['periodStartDate'] >= df['periodStartDate'].max() - pd.DateOffset(weeks=time_window_regr)]

    output = pd.DataFrame()

    for i in business_unit_code:
        df_selected = df_time_selected[df_time_selected['businessUnit'] == i]
    
        platform_code = df_selected['platformCode'].unique().tolist()
        platform_code.sort()

        for z in platform_code:
            temp_dict = dict([('Client', ''), ('Business_unit', ''), ('Platform', ''),
                          ('Corr_coeff', 0), ('Pr_value', 0), ('a', 0), ('b', 0),
                          ('Mean', 0), ('R_std', 0), ('Invest', 0), ('Profit', 0),
                          ('Prof_last_week', 0), ('Invest_last_week', 0),
                          ('Prof_prev_week', 0), ('Invest_prev_week', 0),
                          ('Max_week_profit', 0), ('Max_week_invest', 0)])

            df_platform = df_selected[df_selected['platformCode'] == z]

            df_business_selected_agg = select_agg_resample_df( df_platform, 'periodStartDate', 'week', use_nan=False )
               
            temp_dict['Max_week_profit'] = df_business_selected_agg.grossProfit[:].max()
            temp_dict['Max_week_invest'] = df_business_selected_agg.marketingInvestment[:].max()
        
            temp_dict['Prof_prev_week'] = df_business_selected_agg.grossProfit[-2:-1].max()
            temp_dict['Invest_prev_week'] = df_business_selected_agg.marketingInvestment[-2:-1].max()
        
            temp_dict['Prof_last_week'] = df_business_selected_agg.grossProfit[-1].max()
            temp_dict['Invest_last_week'] = df_business_selected_agg.marketingInvestment[-1].max()
        
            df_business_selected_agg = df_business_selected_agg.iloc[:-1, :] # get rid of last week in dataset     
        
            if filter_type.value == 'AWS':
                df_business_selected_agg_filter = filter_df( df_business_selected_agg, aws_alpha )
            
            elif filter_type.value == 'None':
                df_business_selected_agg_filter = df_business_selected_agg
            
            elif filter_type.value == 'Median':
                print("Median Filter have Not implemented yet") 
        
            else:
                print("Incorrect Filter settings") 

            temp_dict['Client'] = account_code[0]
            temp_dict['Business_unit'] = i
            temp_dict['Platform'] = z

            temp_dict['Profit'] = np.round( df_business_selected_agg['grossProfit'].sum(), 2 )
            temp_dict['Invest'] = np.round( df_business_selected_agg['marketingInvestment'].sum(), 2)

            sper_crr, P_value = stat.spearmanr( df_business_selected_agg_filter['marketingInvestment'],                       df_business_selected_agg_filter['grossProfit'])

            temp_dict['Corr_coeff'] = sper_crr
            temp_dict['Pr_value'] = P_value
        
        #sorted_df = df_business_selected_agg_filter.sort_values( by=['marketingInvestment'], ascending=False )
            sorted_df = df_business_selected_agg_filter
        
            temp_dict['Mean'] = sorted_df['grossProfit'].describe()[1]
        
            if sper_crr > corr_thld:
                print(i + ' business unit ' + z + ' platform has \t' + str(sper_crr) + ' corr coefficient')
                try:
                    t_train = sorted_df['marketingInvestment']
                    y_train = sorted_df['grossProfit']

                    regr, err = regression_calc(sorted_df[['marketingInvestment', 'grossProfit']], log_f)

                    R = sorted_df['grossProfit'] - \
                    log_f(sorted_df['marketingInvestment'], *regr)
                    R_std = R.describe()['std']

                    a, b = regr
                    temp_dict['a'] = a
                    temp_dict['b'] = b
                    temp_dict['R_std'] = R_std
                
                    #print('Regression calculated successfully')
                except:
                    print("Regression wasn't calculated")
                    temp_dict['a'] = 0
                    temp_dict['b'] = 0
            else:
                temp_dict['a'] = 0
                temp_dict['b'] = 0

            output = output.append(temp_dict, ignore_index=True)
    return output

In [105]:
list_files = os.listdir("./input")
mypath = Path().absolute()
folder = 'budget_opt\\'
img_folder = folder + 'img\\'
corr_thld = 0.9
aws_alpha = 0.1
time_window_regr = 12

## Read out dataset form CSV file

In [522]:
file_name = widgets.Dropdown(
    options = list_files,
    value=list_files[0],
    description='Source file:',
    disabled=False,
)
display(file_name)

Dropdown(description='Source file:', options=('4Camping.csv', '4home.csv', 'aliatic.csv', 'apotek.csv', 'audio…

In [525]:
df = pd.read_csv('input\\'+ file_name.value, sep="\t", infer_datetime_format=True,
                 usecols=usecols_, dtype=dtype_, parse_dates=DateTimeID)

df['periodStartDate'] = pd.to_datetime(df['periodStartDate'])
df.head(5)

,account,businessUnit,platformCode,channelCode,adCampaign,marketingInvestment,impressions,clicks,visits,conversions,deliveries,netRevenue,grossProfit,currencyCode,adGroup,periodStartDate
0,forcamping,forcampingcz,unpaid,notset,NaN,0.0,0,0,0,5,5,10724.79,10724.79,CZK,-,2020-01-21
1,forcamping,forcampingcz,unpaid,direct,NaN,0.0,0,0,0,22,22,48687.03,48687.03,CZK,-,2020-01-21
2,forcamping,forcampingcz,HeurekaCz,heureka,NaN,0.0,0,0,51,2,2,3342.98,3342.98,CZK,-,2020-01-21
3,forcamping,forcampingcz,unpaid,organic,NaN,0.0,0,0,0,59,59,121605.97,121605.97,CZK,-,2020-01-21
4,forcamping,forcampingcz,ZboziCz,zbozi,NaN,0.0,0,0,83,3,3,3604.96,3604.96,CZK,-,2020-01-21


## Main settings

In [8]:
# Data source
data_source = widgets.Dropdown(
    options = ['platform','compaign','channel'],
    value='platform',
    description='Data source:',
    disabled=False,
)
display(data_source)

Dropdown(description='Data source:', options=('platform', 'compaign', 'channel'), value='platform')

In [9]:
# Data source
filter_type = widgets.Dropdown(
    options = ['None','AWS','Median'],
    value='AWS',
    description='Filter type:',
    disabled=False,
)
display(filter_type)

Dropdown(description='Filter type:', index=1, options=('None', 'AWS', 'Median'), value='AWS')

In [10]:
# Data source
AWS_value = widgets.Dropdown(
    options = [0.05,0.1,0.15,0.2],
    value=0.1,
    description='AWS filter coeff:',
    disabled=False,
)
display(AWS_value)

Dropdown(description='AWS filter coeff:', index=1, options=(0.05, 0.1, 0.15, 0.2), value=0.1)

In [114]:
settings_list = ['None','Correlation','Optimization','Both']
save_settings = widgets.Dropdown(
    options = settings_list,
    value = settings_list[0],
    description='Save to file:',
    disabled=False,
)
display(save_settings)

Dropdown(description='Save to file:', options=('None', 'Correlation', 'Optimization', 'Both'), value='None')

In [116]:
log_list = ['All','None','Error only']
log_settings = widgets.Dropdown(
    options = log_list,
    value = log_list[0],
    description='Log output:',
    disabled=False,
)
display(log_settings)

Dropdown(description='Log output:', options=('All', 'None', 'Error only'), value='All')

In [128]:
Corr_value = widgets.Dropdown(
    options = [0.99,0.95,0.9,0.85],
    value=0.9,
    description='AWS filter coeff:',
    disabled=False,
)
display(Corr_value)

Dropdown(description='AWS filter coeff:', index=2, options=(0.99, 0.95, 0.9, 0.85), value=0.9)

In [220]:
data_source.value

'platform'

## Data processing: 
* aggregating:
* correlation calculation:
* regression calculation:

In [541]:
## function prototype was extracted form here
# select window for regression

business_unit_code = df['businessUnit'].unique().tolist()
business_unit_code.sort()
channel_code_code = df['channelCode'].unique().tolist()
channel_code_code.sort()
currency_code_code = df['currencyCode'].unique().tolist()
currency_code_code.sort()
account_code = df['account'].unique().tolist()

# selecl "time_window_regr" number of last weeks from dataset 

df_time_selected = df[df['periodStartDate'] >= df['periodStartDate'].max() - pd.DateOffset(weeks=time_window_regr)]

output = pd.DataFrame()

for i in business_unit_code:
    
    df_selected = df_time_selected[df_time_selected['businessUnit'] == i]
    
    if data_source.value == 'platform':
        component_code = df_selected['platformCode'].unique().tolist()
        
    elif data_source.value == 'compaign':
        component_code = df_selected['adCampaign'].unique().tolist()
        
        
    elif data_source.value == 'channel':
        component_code = df_selected['channelCode'].unique().tolist()
       
    
    else:
        print('Incorrect component settings')
        
    #component_code.sort()

    for z in component_code:
        temp_dict = dict([('Client', ''), ('Business_unit', ''), ('Platform', ''),
                          ('Corr_coeff', 0), ('Pr_value', 0), ('a', 0), ('b', 0),
                          ('Mean', 0), ('R_std', 0), ('Invest', 0), ('Profit', 0),
                          ('Prof_last_week', 0), ('Invest_last_week', 0),
                          ('Prof_prev_week', 0), ('Invest_prev_week', 0),
                          ('Max_week_profit', 0), ('Max_week_invest', 0)])

        
        if data_source.value == 'platform':
            df_platform = df_selected[df_selected['platformCode'] == z]
        
        elif data_source.value == 'compaign':
            df_platform = df_selected[df_selected['adCampaign'] == z]
        
        
        elif data_source.value == 'channel':
            df_platform = df_selected[df_selected['channelCode'] == z]
        
        
        

        df_business_selected_agg = select_agg_resample_df( df_platform, 'periodStartDate', 'week', use_nan=False )
               
        temp_dict['Max_week_profit'] = df_business_selected_agg.grossProfit[:].max()
        temp_dict['Max_week_invest'] = df_business_selected_agg.marketingInvestment[:].max()
        
        temp_dict['Prof_prev_week'] = df_business_selected_agg.grossProfit[-2:-1].max()
        temp_dict['Invest_prev_week'] = df_business_selected_agg.marketingInvestment[-2:-1].max()
        
        temp_dict['Prof_last_week'] = df_business_selected_agg.grossProfit[-1].max()
        temp_dict['Invest_last_week'] = df_business_selected_agg.marketingInvestment[-1].max()
        
        df_business_selected_agg = df_business_selected_agg.iloc[:-1, :] # get rid of last week in dataset     
        
        if filter_type.value == 'AWS':
            df_business_selected_agg_filter = filter_df( df_business_selected_agg, aws_alpha )
            
        elif filter_type.value == 'None':
            df_business_selected_agg_filter = df_business_selected_agg
            
        elif filter_type.value == 'Median':
            print("Median Filter have Not implemented yet") 
        
        else:
            print("Incorrect Filter settings") 

        temp_dict['Client'] = account_code[0]
        temp_dict['Business_unit'] = i
        temp_dict['Component'] = z

        temp_dict['Profit'] = np.round( df_business_selected_agg['grossProfit'].sum(), 2 )
        temp_dict['Invest'] = np.round( df_business_selected_agg['marketingInvestment'].sum(), 2)

        sper_crr, P_value = stat.spearmanr( df_business_selected_agg_filter['marketingInvestment'],                       df_business_selected_agg_filter['grossProfit'])

        temp_dict['Corr_coeff'] = sper_crr
        temp_dict['Pr_value'] = P_value
        
        #sorted_df = df_business_selected_agg_filter.sort_values( by=['marketingInvestment'], ascending=False )
        sorted_df = df_business_selected_agg_filter
        
        temp_dict['Mean'] = sorted_df['grossProfit'].describe()[1]
        
        if sper_crr > Corr_value.value:
            #print(i + ' business unit ' + z + ' platform has \t' + str(sper_crr) + ' corr coefficient')
            try:
                t_train = sorted_df['marketingInvestment']
                y_train = sorted_df['grossProfit']

                regr, err = regression_calc(sorted_df[['marketingInvestment', 'grossProfit']], log_f)

                R = sorted_df['grossProfit'] - \
                    log_f(sorted_df['marketingInvestment'], *regr)
                R_std = R.describe()['std']

                a, b = regr
                temp_dict['a'] = a
                temp_dict['b'] = b
                temp_dict['R_std'] = R_std
                
                #print('Regression calculated successfully')
            except:
                print("Regression wasn't calculated")
                temp_dict['a'] = 0
                temp_dict['b'] = 0
        else:
            temp_dict['a'] = 0
            temp_dict['b'] = 0

        output = output.append(temp_dict, ignore_index=True)

filterd_out = output[~output['Platform'].isin(dropped_ch)]
corr_df = filterd_out[filterd_out['Corr_coeff'] > corr_thld]
non_corr_df = filterd_out[filterd_out['Corr_coeff'] <= corr_thld]
control_inv = corr_df['Invest'].sum()
non_control_inv = non_corr_df['Invest'].sum()
control_prof = corr_df['Profit'].sum()
non_control_iprof = non_corr_df['Profit'].sum()

In [542]:
#output = platform_calc(df) # it doesn't work

1. select last week and calculate total investment and total profit
2. plot regression in range up to max investment within last 12 weeks / plot point - last week
3. add to dataframe: last week invest, profit, and best week invest, profit
4. Optimization  - change max 10% of 
5. print out list or recommendations
6. print out top 10 or list of components form 85% of budget/profit

## Processing results

In [543]:
#display(filterd_out.head(5))
print('The total number of compomnents equal '  + str(len(filterd_out)))

corr_thd_df = filterd_out[filterd_out['Corr_coeff'] >= Corr_value.value]
corr_thd_df = corr_thd_df[corr_thd_df['Prof_last_week'] > 0]
corr_thd_df = corr_thd_df[corr_thd_df['Prof_prev_week'] > 0]
corr_thd_df.reset_index(inplace=True, drop=True)
corr_thd_df.head(20)
print("The number of components with correlation >= threshold equal " + str(len(corr_thd_df)))
print(f"All investment for last 12 weeks are {filterd_out['Invest'].sum()}")
print(f"All controled investment for last 12 weeks are {corr_thd_df['Invest'].sum()}")



The total number of compomnents equal 31


,Business_unit,Client,Component,Corr_coeff,Invest,Invest_last_week,Invest_prev_week,Max_week_invest,Max_week_profit,Mean,Platform,Pr_value,Prof_last_week,Prof_prev_week,Profit,R_std,a,b
0,forcampingcz,forcamping,notset,0.972028,317365.40,36991.09,33280.19,102401.64,695648.12,224131.020611,,1.286812e-07,404637.21,442612.94,5022440.54,11529.868264,3.164712e+05,5.935032e+03
1,forcampingcz,forcamping,criteo,0.993007,67789.67,4956.18,6047.11,6320.67,132136.11,55405.222527,,1.301673e-10,113442.16,107817.76,1142152.95,1836.283516,1.227312e+05,4.747344e+03
2,forcampingcz,forcamping,heureka,0.979021,222673.13,30636.39,30244.50,33132.28,1015661.07,321917.633458,,3.089801e-08,760150.39,838491.14,8068447.30,23257.777361,1.273541e+06,2.672932e+04
3,forcampingcz,forcamping,zbozi,0.979021,34583.40,4118.10,3783.30,4166.10,352958.63,110627.769487,,3.089801e-08,231206.37,259138.78,2629182.95,5745.758200,-6.796166e+05,-9.718661e+03
4,forcampingcz,forcamping,Sklik-BRAND,0.937063,5061.04,670.00,578.80,735.94,263101.39,65027.541243,,6.993165e-06,208705.92,195910.09,1729166.30,4187.198533,-4.445387e+08,-1.249775e+06
5,forcampingcz,forcamping,cjaffiliate,0.906392,103494.52,26286.72,32137.67,32137.67,480485.11,124430.342271,,4.828598e-05,290653.15,347712.57,3046087.55,37215.479773,1.752522e+05,3.187368e+02
6,forcampingcz,forcamping,Sklik-SEA-BRA,0.993007,94567.65,12637.23,9584.66,12637.23,107689.01,31575.627216,,1.301673e-10,78458.46,79791.33,823703.82,1811.729702,-2.368605e+05,-2.966269e+04
7,forcampingcz,forcamping,Ads-SEA-BRA,1.000000,164639.48,27702.60,25983.93,27702.60,262878.28,33627.092994,,0.000000e+00,220411.74,262878.28,1184629.94,3961.306837,-1.137364e+05,-2.138192e+04
8,forcampingcz,forcamping,Sklik-SEA-GEN,0.979021,42683.63,4703.57,4891.55,5144.02,52542.12,13302.514293,,3.089801e-08,23698.14,52542.12,329902.92,844.581374,-1.052270e+08,-1.403281e+07
9,forcampingcz,forcamping,Ads-SEA-GEN,1.000000,326666.87,38068.02,35816.30,63073.15,180177.26,32800.000698,,0.000000e+00,162476.34,180177.26,960498.62,4148.889594,-6.982375e+04,-3.421143e+04


The number of components with correlation >= threshold equal 15
All investment for last 12 weeks are 3818675.2600000002
All controled investment for last 12 weeks are 1716343.4599999997


## Visual inspection (sanity check)

In [545]:
zzz_list =  [i for i in range(len(corr_thd_df))]

In [476]:

data_line = widgets.Dropdown(
    options = zzz_list,
    value=zzz_list[0],
    description='Source file:',
    disabled=False,
)
display(data_line)

Dropdown(description='Source file:', options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12), value=0)

In [547]:
zzz_df = corr_thd_df.iloc[data_line.value:data_line.value + 1, :]
zzz_df
show_chart(zzz_df)

,Business_unit,Client,Component,Corr_coeff,Invest,Invest_last_week,Invest_prev_week,Max_week_invest,Max_week_profit,Mean,Platform,Pr_value,Prof_last_week,Prof_prev_week,Profit,R_std,a,b
1,forcampingcz,forcamping,criteo,0.993007,67789.67,4956.18,6047.11,6320.67,132136.11,55405.222527,,1.301673e-10,113442.16,107817.76,1142152.95,1836.283516,122731.247584,4747.344327


## Budget optimization
 Settings:
* Aggressive 20% max change
* Conservative 10% max change

In [397]:
optimization_settings_list = ['Conservative','Aggressive']
optimization_settings = widgets.Dropdown(
    options = optimization_settings_list,
    value = optimization_settings_list[0],
    description='Optimization level:',
    disabled=False,
)
display(optimization_settings)

Dropdown(description='Optimization level:', options=('Conservative', 'Aggressive'), value='Conservative')

In [398]:
optimization_type_list = ['Keep budget','Change budget']
optimization_type = widgets.Dropdown(
    options = optimization_type_list,
    value = optimization_type_list[0],
    description='Optimization type:',
    disabled=False,
)
display(optimization_type)

Dropdown(description='Optimization type:', options=('Keep budget', 'Change budget'), value='Keep budget')

In [399]:
number = widgets.BoundedIntText(
    value=0,
    min=-1000,
    max=1000,
    step=100,
    description='Budget change:',
    disabled=False
)
display(number)

BoundedIntText(value=0, description='Budget change:', max=1000, min=-1000, step=100)

In [531]:
#corr_thd_df.head(5)

### Budget optimizer start here

In [548]:
# budget optimizer preprocessing 
corr_thd_df['+10%invest'] = corr_thd_df['Invest_prev_week'] * 1.1
corr_thd_df['+20%invest'] = corr_thd_df['Invest_prev_week'] * 1.2
corr_thd_df['-10%invest'] = corr_thd_df['Invest_prev_week'] * 0.9
corr_thd_df['-20%invest'] = corr_thd_df['Invest_prev_week'] * 0.8

corr_thd_df['slope_plus_10'] = 0.0
corr_thd_df['slope_plus_20'] = 0.0
corr_thd_df['slope_minus_10'] = 0.0
corr_thd_df['slope_minus_20'] = 0.0

corr_thd_df['opt_invest'] = 0.0
corr_thd_df['profit_change'] = 0.0

for i in range(len(corr_thd_df)):
    profit_base = float(log_f(corr_thd_df['Invest_prev_week'][i], corr_thd_df['a'][i], corr_thd_df['b'][i]))
    
    profit_plus_10 = float(log_f(corr_thd_df['+10%invest'][i], corr_thd_df['a'][i], corr_thd_df['b'][i]))
    slope_plus_10 = float((profit_plus_10 - profit_base))/float((corr_thd_df['+10%invest'][i] - corr_thd_df['Invest_prev_week'][i]))
    corr_thd_df['slope_plus_10'][i] = slope_plus_10
    
    profit_plus_20 = float(log_f(corr_thd_df['+20%invest'][i], corr_thd_df['a'][i], corr_thd_df['b'][i]))
    slope_plus_20 = float((profit_plus_20 - profit_base))/float((corr_thd_df['+20%invest'][i] - corr_thd_df['Invest_prev_week'][i]))
    corr_thd_df['slope_plus_20'][i] = slope_plus_20
    
    profit_minus_10 = float(log_f(corr_thd_df['-10%invest'][i], corr_thd_df['a'][i], corr_thd_df['b'][i]))
    slope_minus_10 = float((profit_minus_10 - profit_base))/float((corr_thd_df['-10%invest'][i] - corr_thd_df['Invest_prev_week'][i]))
    corr_thd_df['slope_minus_10'][i] = -slope_minus_10
    
    profit_minus_20 = float(log_f(corr_thd_df['-20%invest'][i], corr_thd_df['a'][i], corr_thd_df['b'][i]))
    slope_minus_20 = float((profit_minus_20 - profit_base))/float((corr_thd_df['-20%invest'][i] - corr_thd_df['Invest_prev_week'][i]))
    corr_thd_df['slope_minus_20'][i] = -slope_minus_20
    
invest_pool_plus_10 = (corr_thd_df['Invest_prev_week'] - corr_thd_df['-10%invest']).sum()


invest_pool_plus_20 = (corr_thd_df['Invest_prev_week'] - corr_thd_df['-20%invest']).sum()


In [549]:
zzz = corr_thd_df.sort_values(by=['slope_plus_10'], ascending=False)
zzz.reset_index(inplace=True, drop=True)
#zzz

In [550]:
if optimization_type.value == 'Keep budget':
    if optimization_settings.value == 'Conservative':
        accum = invest_pool_plus_10
    elif optimization_settings.value == 'Aggressive':
        accum = invest_pool_plus_20
elif optimization_type.value == 'Change budget':
    if optimization_settings.value == 'Conservative':
        accum = invest_pool_plus_10 + number.value
    elif optimization_settings.value == 'Aggressive':
        accum = invest_pool_plus_20 + number.value
        
        
for i in range (len(zzz)):
    temp = (zzz['+10%invest'][i] - zzz['-10%invest'][i] )
    if temp <= accum:
        zzz['opt_invest'][i] = zzz['+10%invest'][i]
        accum = accum - temp
    else:
        zzz['opt_invest'][i] = zzz['-10%invest'][i] + accum
        accum = 0.0       

In [551]:
#zzz

In [552]:
for i in range (len(zzz)):
    profit_base = float(log_f(zzz['Invest_prev_week'][i], zzz['a'][i], zzz['b'][i]))
    optim_profit = float(log_f(zzz['opt_invest'][i], zzz['a'][i], zzz['b'][i]))
    zzz['profit_change'][i] = optim_profit - profit_base
    
zzz['Invest_prev_week'].sum()
zzz['opt_invest'].sum()
zzz['Prof_prev_week'].sum()
zzz['profit_change'].sum()

222653.84000000003

222653.84000000005

3088037.5700000003

194107.13829617694

In [553]:
zzz

,Business_unit,Client,Component,Corr_coeff,Invest,Invest_last_week,Invest_prev_week,Max_week_invest,Max_week_profit,Mean,Platform,Pr_value,Prof_last_week,Prof_prev_week,Profit,R_std,a,b,+10%invest,+20%invest,-10%invest,-20%invest,slope_plus_10,slope_plus_20,slope_minus_10,slope_minus_20,opt_invest,profit_change
0,forcampingcz,forcamping,Sklik-BRAND,0.937063,5061.04,670.00,578.80,735.94,263101.39,65027.541243,,6.993165e-06,208705.92,195910.09,1729166.30,4187.198533,-4.445387e+08,-1.249775e+06,636.680,694.560,520.920,463.040,355.868007,355.876247,-355.851526,-355.843286,636.680,20597.640224
1,forcampingcz,forcamping,Sklik-SEA-PRO,0.902098,16134.05,1416.37,1544.38,2504.83,47450.98,11570.290841,,5.997857e-05,22988.03,15114.38,275884.98,1531.989122,-5.198974e+03,-6.317619e+02,1698.818,1853.256,1389.942,1235.504,107.445168,122.322375,-84.143582,-75.019491,1698.818,16593.616907
2,forcampingcz,forcamping,zbozi,0.979021,34583.40,4118.10,3783.30,4166.10,352958.63,110627.769487,,3.089801e-08,231206.37,259138.78,2629182.95,5745.758200,-6.796166e+05,-9.718661e+03,4161.630,4539.960,3404.970,3026.640,105.244922,107.333814,-101.226646,-99.294217,4161.630,39817.311296
3,forcampingcz,forcamping,Sklik-DIS,0.943860,10162.41,429.55,720.38,2607.25,20452.56,2321.594746,,3.995310e-06,4015.04,20200.22,87556.21,369.018580,-7.448873e+02,-2.944551e+02,792.418,864.456,648.342,576.304,33.095864,37.682493,-25.913390,-23.101524,792.418,2384.159841
4,forcampingcz,forcamping,Ads-SEA-BRA,1.000000,164639.48,27702.60,25983.93,27702.60,262878.28,33627.092994,,0.000000e+00,220411.74,262878.28,1184629.94,3961.306837,-1.137364e+05,-2.138192e+04,28582.323,31180.716,23385.537,20787.144,19.066703,20.298558,-16.884915,-15.918851,28582.323,49542.788760
5,forcampingcz,forcamping,Ads-DREM,0.958042,34923.65,6556.72,4648.69,6556.72,88454.06,26106.023307,,9.543582e-07,76865.99,33735.02,521027.71,3037.192397,9.660166e+07,5.859882e+06,5113.559,5578.428,4183.821,3718.952,16.471529,16.470876,-16.472836,-16.473490,5113.559,7657.103379
6,forcampingcz,forcamping,heureka,0.979021,222673.13,30636.39,30244.50,33132.28,1015661.07,321917.633458,,3.089801e-08,760150.39,838491.14,8068447.30,23257.777361,1.273541e+06,2.672932e+04,33268.950,36293.400,27220.050,24195.600,14.530400,13.753138,-16.271157,-17.245808,33268.950,43946.467338
7,forcampingcz,forcamping,Ads-DSA-PRO,0.937063,82776.37,10115.30,9868.98,13355.34,140809.18,40576.429791,,6.993165e-06,126410.38,109456.65,897240.21,5558.798435,-8.662412e+08,-7.136667e+07,10855.878,11842.776,8882.082,7895.184,12.139658,12.139742,-12.139490,-12.139406,10855.878,11980.604347
8,forcampingcz,forcamping,Sklik-SEA-BRA,0.993007,94567.65,12637.23,9584.66,12637.23,107689.01,31575.627216,,1.301673e-10,78458.46,79791.33,823703.82,1811.729702,-2.368605e+05,-2.966269e+04,10543.126,11501.592,8626.194,7667.728,11.211082,11.395167,-10.854618,-10.682053,10543.126,10745.441018
9,forcampingcz,forcamping,Ads-SEA-PRO,0.993007,192822.19,22310.46,23523.40,23769.07,168002.21,58616.910886,,1.301673e-10,159716.42,142459.03,1428249.09,6407.002649,-1.138563e+09,-1.443916e+08,25875.740,28228.080,21171.060,18818.720,7.886596,7.886660,-7.886468,-7.886403,25875.740,18551.955406


limit_output extension: Maximum message size of 10000 exceeded with 12237 characters

## Save report to file

In [556]:
name = file_name.value
file_name_str = name.split('.')[0] + f' on {data_source.value} level optimization_report.xlsx' 

with pd.ExcelWriter(file_name_str) as writer:
    zzz.to_excel(writer)

'channel'